In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from time import time
from cv2 import VideoCapture
import math

In [2]:
# faz a leitura de video e guarda todos os frames num array
def load_frames(videoPath):
    cap = VideoCapture(videoPath)
    start = time()

    
    frameArray = []
    print("Collecting frame data")
    while 1:
        ret, frame = cap.read()
        if frame is None:
            print("Frame Data Collected")
            break

        target = frame[60:630, 380:1100]
        frameArray.append(target)
        
        if cv2.waitKey(1) == 27 or cv2.waitKey(1) == 113 or cv2.waitKey(1) == 81:
            cap.release()
            exit(0)
            break


    
    end = time()
    print("Time spent collecting : " + str(end-start))
    print("Amount of frames collected : "+ str(len(frameArray)))
    return frameArray   

Frame Data Collected
Time spent collecting : 37.00911521911621
Amount of frames collected : 2827


In [3]:
# realiza o método otsu equivalente cv2.threshold com a flag cv2.THRESH_OTSU no entanto é possivel obter
# métricas que não são possiveis na função do OpenCV como
def otsuThresh(components):
    i= 0
    bins_num = [180,255,255]
    max_val = 0
    thres = []
    indexc = 0
    for c in components:
        # Get the image histogram
        hist, bin_edges = np.histogram(c[400,:], bins=bins_num[i])
        
        # Get normalized histogram if it is required

        hist = np.divide(hist.ravel(), hist.max())

        # Calculate centers of bins
        bin_mids = (bin_edges[:-1] + bin_edges[1:]) / 2.

        # Iterate over all thresholds (indices) and get the probabilities w1(t), w2(t)
        weight1 = np.cumsum(hist)
        weight2 = np.cumsum(hist[::-1])[::-1]

        # Get the class means mu0(t)
        mean1 = np.cumsum(hist * bin_mids) / weight1
        # Get the class means mu1(t)
        mean2 = (np.cumsum((hist * bin_mids)[::-1]) / weight2[::-1])[::-1]

        inter_class_variance = weight1[:-1] * weight2[1:] * (mean1[:-1] - mean2[1:]) ** 2

        # Maximize the inter_class_variance function val
        index_of_max_val = np.argmax(inter_class_variance)
        #print(np.max(inter_class_variance))
        if(max_val < np.max(inter_class_variance)):
            max_val = np.max(inter_class_variance)
            indexc = i
            
        i += 1
        threshold = bin_mids[:-1][index_of_max_val]
        #print("Otsu's algorithm implementation thresholding result: ", threshold)
        thres.append(threshold)
    return thres,indexc

In [4]:
def convertScale(img, alpha, beta):
    """Add bias and gain to an image with saturation arithmetics. Unlike
    cv2.convertScaleAbs, it does not take an absolute value, which would lead to
    nonsensical results (e.g., a pixel at 44 with alpha = 3 and beta = -210
    becomes 78 with OpenCV, when in fact it should become 0).
    """

    new_img = img * alpha + beta
    new_img[new_img < 0] = 0
    new_img[new_img > 255] = 255
    return new_img.astype(np.uint8)

# Automatic brightness and contrast optimization with optional histogram clipping
def automatic_brightness_and_contrast(image, clip_hist_percent=25):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Calculate grayscale histogram
    hist = cv2.calcHist([gray],[0],None,[256],[0,256])
    hist_size = len(hist)

    # Calculate cumulative distribution from the histogram
    accumulator = []
    accumulator.append(float(hist[0]))
    for index in range(1, hist_size):
        accumulator.append(accumulator[index -1] + float(hist[index]))

    # Locate points to clip
    maximum = accumulator[-1]
    clip_hist_percent *= (maximum/100.0)
    clip_hist_percent /= 2.0

    # Locate left cut
    minimum_gray = 0
    while accumulator[minimum_gray] < clip_hist_percent:
        minimum_gray += 1

    # Locate right cut
    maximum_gray = hist_size -1
    while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
        maximum_gray -= 1

    # Calculate alpha and beta values
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = -minimum_gray * alpha


In [5]:
def adjust_gamma(image, gamma=1.0):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
	invGamma = 1.0 / gamma
	table = np.array([((i / 255.0) ** invGamma) * 255
		for i in np.arange(0, 256)]).astype("uint8")
	# apply gamma correction using the lookup table
	return cv2.LUT(image, table)

In [6]:
def calibration(videoPath,gamma = 1.0):
    cap = VideoCapture(videoPath)
    change = False
    while 1:
        ret, frame = cap.read()
        if frame is None:
            break

        target = frame[60:630, 380:1100]
        
        h = len(target)
        start_height = h - 30
      
        target = adjust_gamma(target,gamma)
        hsv = cv2.cvtColor(target,cv2.COLOR_BGR2HSV)
        hue,saturation,value = cv2.split(hsv)
        components = [hue,saturation,value]
        thresholds,index = otsuThresh(components)
        
        if(index == 2):
            grayFrame = cv2.cvtColor(target,cv2.COLOR_BGR2GRAY)
            guassianFrame = cv2.GaussianBlur(grayFrame,(5,5),cv2.BORDER_DEFAULT)
            ret,thresh1 = cv2.threshold(guassianFrame,0 , 255 , cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        else:
            guassianFrame = cv2.GaussianBlur(components[index],(5,5),cv2.BORDER_DEFAULT)
            ret,thresh1 =  cv2.threshold(guassianFrame,0 , 255 , cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            
        no_points_count = 0
        
        middlePixel = len(target[0])/2
        
        while(start_height > 0): 
            signed_thresh = thresh1[start_height].astype(np.int16) # select only one row
            diff = np.diff(signed_thresh)   #The derivative of the start_height line

            points = np.where(np.logical_or(diff > 200, diff < -200))

            if len(points) > 0 and len(points[0]) > 1:
                
                middle = int((points[0][0] + points[0][1]) / 2)
                
                if((len(target[0])/2) - (middlePixel/10) < middle
                   <= (len(target[0])/2) + (middlePixel/10) and start_height == h - 30):
                    change = True
                if( (len(target[0])/2) - (middlePixel/10)< middle
                   <= (len(target[0])/2) + (middlePixel/10) and  change == True):
                    color = (0,255,0)
                    
                if(change == False):
                    color = (0,0,255)    
                    
                cv2.circle(target, (points[0][0], start_height), 5, (255,0,0), -1)
                cv2.circle(target, (points[0][1], start_height), 5, (255,0,0), -1)
                cv2.circle(target, (middle, start_height), 5, color, -1)
                start_height -= 5
                start_height = start_height % h
            else:
                start_height -= 5
                start_height = start_height % h        
                no_points_count += 1
        """
        #CASO SE QUEIRA OBSERVAR AS IMAGENS DE CALIBRAÇÃO FAVOR DESCOMENTAR ESTE BLOCO:
        plt.imshow(cv2.cvtColor(target,cv2.COLOR_BGR2RGB))
        plt.show()
        """
        """
        #E CASO QUEIRA OBSERVAR AS IMAGENS NUMA JANELA À PARTE DESCOMENTAR ESTE BLOCO:
        cv2.imshow("calibration Video",target)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        """
        
        change = False
        
        if cv2.waitKey(1) == 27 or cv2.waitKey(1) == 113 or cv2.waitKey(1) == 81:
            cap.release()
            exit(0)
            
            return index
    cap.release()
    exit(0)
    return index

In [7]:
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,500)
fontScale              = 2
fontColor              = (0,0,255)
lineType               = 2


def lineAvaliation(image,middleLine):
    middlePixel = len(image[0])/2
    if(middlePixel < middleLine):
        classification = 20 - round(middleLine / (middlePixel/10))
    else:
        classification = round(middleLine / (middlePixel/10))
    
        
    return classification


def processVideo(videoPath,index):
    cap = VideoCapture(videoPath)
    avaliations = []
    while 1:
        ret, frame = cap.read()
        if frame is None:
            break

        target = frame[60:630, 380:1100]
        
        h = len(target)
        start_height = h - 30

        if(index == 2):
            grayFrame = cv2.cvtColor(target,cv2.COLOR_BGR2GRAY)
            ret,thresh1 = cv2.threshold(grayFrame,0 , 255 , cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        else:
            hsv = cv2.cvtColor(target,cv2.COLOR_BGR2HSV)
            hue,saturation,value = cv2.split(hsv)
            components = [hue,saturation,value]
            ret,thresh1 = cv2.threshold(grayFrame,0 , 255 , cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        no_points_count = 0

        while(start_height > 0): 
            signed_thresh = thresh1[start_height].astype(np.int16) # select only one row
            diff = np.diff(signed_thresh)   #The derivative of the start_height line
            
            points = np.where(np.logical_or(diff > 200, diff < -200))

            if len(points) > 0 and len(points[0]) > 1:
                
                middle = int((points[0][0] + points[0][1]) / 2)
                value = None
                if(start_height == h - 30):
                    value = lineAvaliation(thresh1,middle)
                    avaliations.append(value)
                    
                cv2.circle(target, (points[0][0], start_height), 5, (255,0,0), -1)
                cv2.circle(target, (points[0][1], start_height), 5, (255,0,0), -1)
                cv2.circle(target, (middle, start_height), 5, (0,0,255), -1)
                if(value != None):
                    cv2.putText(target,str(value), 
                        bottomLeftCornerOfText, 
                        font, 
                        fontScale,
                        fontColor,
                        lineType)
                start_height -= 5
                start_height = start_height % h
            else:
                start_height -= 5
                start_height = start_height % h        
                no_points_count += 1
        
        plt.imshow(cv2.cvtColor(target,cv2.COLOR_BGR2RGB))
        plt.show()
        """
        cv2.imshow("avaluation Video",target)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        """
        if cv2.waitKey(1) == 27 or cv2.waitKey(1) == 113 or cv2.waitKey(1) == 81:
            cap.release()
            exit(0)
            break
    
    
    cap.release()
    exit(0)
    return round(np.mean(avaliations))
    
index = calibration('./VideoContent/boa_amostra_preto3.mp4',1.0)
print('calibration Done')
processVideo('./VideoContent/muito_boa_amostra_preto_pouco_amarelo.mp4',index)

calibration Done


8